In [1]:
import pandas as pd

In [2]:
df=pd.read_excel(r"data\raw\RIESGO.xlsx")

In [3]:
df.head(5)

,ID,EDAD,INGRE,GENERO,ECIVIL,HIJOS,NUMTDC,FPAGO,HIPOTECA,TIPCRED,CREDITOS,RIESGO
0,100756,44.0,59944.0,m,Casado,1,2,Mensual,s,1,0,Bueno
1,100668,35.0,59692.0,m,Casado,1,1,Mensual,s,1,0,Malo
2,100418,34.0,59508.0,m,Casado,1,1,Mensual,s,1,1,Bueno
3,100416,34.0,59463.0,m,Casado,0,2,Mensual,s,1,1,Malo
4,100590,39.0,59393.0,f,Casado,0,2,Mensual,s,1,0,Bueno


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        4117 non-null   int64  
 1   EDAD      4117 non-null   float64
 2   INGRE     4116 non-null   float64
 3   GENERO    4116 non-null   object 
 4   ECIVIL    4116 non-null   object 
 5   HIJOS     4117 non-null   int64  
 6   NUMTDC    4117 non-null   int64  
 7   FPAGO     4115 non-null   object 
 8   HIPOTECA  4116 non-null   object 
 9   TIPCRED   4117 non-null   int64  
 10  CREDITOS  4117 non-null   int64  
 11  RIESGO    4117 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 386.1+ KB


In [5]:
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

fig = ff.create_table(df.isnull().sum().to_frame(name="Count"), index=True)
miss_values = df.isnull().sum().to_dict()
fig.add_trace(
    go.Bar(
        x=list(miss_values.keys()),
        y=list(miss_values.values()),
        xaxis="x2",
        yaxis="y2",
        name="",
    )
)
fig.update_layout(
    title_text="Missing Values",
    height=500,
    width=1000,
    margin={"t": 65, "l": 40,'r':10},
    xaxis={"domain": [0, 0.25]},
    xaxis2={"domain": [0.4, 0.9]},
    yaxis2={"domain": [0, 1], "anchor": "x2", "title": "Count"},
    template="plotly_white",
)
fig.show()
pio.write_image(fig, 
                'report\img/missing_values.png',
                scale=2)

In [6]:
df["RIESGO"].value_counts()

RIESGO
Medio    2407
Malo      905
Bueno     804
Mal         1
Name: count, dtype: int64

In [7]:
import plotly.express as px

target_ratio = df["RIESGO"].value_counts().to_dict()
fig = px.pie(
    names=["Medio", "Malo","Bueno","Mal"],
    values=list(target_ratio.values()),
    title="Proporcion de la variable de interes",
    width=1000,
)

fig.update_layout(
    width=600,
    margin={"t": 65, "l": 40,'r':10},
    template="plotly_white",
)
fig.show()

pio.write_image(fig, 
                'report\img/target_prop.png',
                scale=2)

In [8]:
df['ID'].duplicated().sum()

5

In [9]:
df1 = df[df['ID'].duplicated(keep=False)]

df1 = df1.groupby(df1.columns.tolist()).apply(lambda x: x.index.tolist()).values.tolist()

In [10]:
df1

[[770], [769], [772], [774], [773], [771]]

In [11]:
df.loc[770]

ID             103771
EDAD             32.0
INGRE         29397.0
GENERO              m
ECIVIL      ViuSepDiv
HIJOS               4
NUMTDC              5
FPAGO       Quincenal
HIPOTECA            s
TIPCRED             1
CREDITOS            2
RIESGO          Medio
Name: 770, dtype: object

In [12]:
df.loc[769]

ID           103771
EDAD           40.0
INGRE       29412.0
GENERO            f
ECIVIL       Casado
HIJOS             2
NUMTDC            4
FPAGO       Mensual
HIPOTECA          s
TIPCRED           1
CREDITOS          2
RIESGO         Malo
Name: 769, dtype: object

In [13]:
df_noID=df.drop(columns=["ID"])

In [14]:
df_noID.duplicated().sum()

0

In [15]:
from utils_app import plot_continuos_histogram_matrix, plot_cat_histogram_matrix,show_corr


plot_continuos_histogram_matrix(df,save='report\img/continuos_hist.png')

In [16]:
plot_cat_histogram_matrix(df,save='report\img/cat_hist.png')

Que se encontró?

- Posibles outliers en EDAD, INGRESOS e HIJOS
- TIPCRED es constante en todas las muestras, es una variable en comun que podría ser eliminada
- ID's duplicados
- "rare classes" en ECIVIL
- Variable de interes desbalanceada
- Datos mal etiquetas

Acciones a tomar:

- Lidiar con duplicados? 
  - Afectan al problema que se quiere solucionar?
- Tratamiento de datos nulos (Eliminar filas o hacer imputacion)
- Normalización de las etiquetas (Agrupar etiquetas MAL a MALO)
- Lidiar con las clases raras

Que hacer primero?

- Borrar columna ID's, es irrelevante para el modelado
- Eliminar columnas con datos nulos
- Agrupar categorias 
- Revisar outliers
- RandomForest o XGBoost

In [17]:
data=df.drop(columns=['ID'])

In [18]:
data.head(5)

,EDAD,INGRE,GENERO,ECIVIL,HIJOS,NUMTDC,FPAGO,HIPOTECA,TIPCRED,CREDITOS,RIESGO
0,44.0,59944.0,m,Casado,1,2,Mensual,s,1,0,Bueno
1,35.0,59692.0,m,Casado,1,1,Mensual,s,1,0,Malo
2,34.0,59508.0,m,Casado,1,1,Mensual,s,1,1,Bueno
3,34.0,59463.0,m,Casado,0,2,Mensual,s,1,1,Malo
4,39.0,59393.0,f,Casado,0,2,Mensual,s,1,0,Bueno


In [19]:
data=data.dropna()

In [20]:
data.isnull().sum()

EDAD        0
INGRE       0
GENERO      0
ECIVIL      0
HIJOS       0
NUMTDC      0
FPAGO       0
HIPOTECA    0
TIPCRED     0
CREDITOS    0
RIESGO      0
dtype: int64

In [21]:
data['RIESGO']=data['RIESGO'].replace("Mal","Malo")

In [22]:
target_ratio = data["RIESGO"].value_counts().to_dict()
fig = px.pie(
    names=["Medio", "Malo","Bueno"],
    values=list(target_ratio.values()),
    title="Proporcion de la variable de interes",
    width=1000,
)
fig.update_layout(
    width=600,
    margin={"t": 65, "l": 40,'r':10},
    template="plotly_white",
)
fig.show()

pio.write_image(fig, 
                'report\img/target_prop_grouped.png',
                scale=2)

In [23]:
data['ECIVIL']=data['ECIVIL'].replace("Soltera","Soltero")
data['ECIVIL']=data['ECIVIL'].replace("Casada","Casado")

In [24]:
import numpy as np 

Q1 = data['EDAD'].quantile(0.25)
Q3 = data['EDAD'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

print(f"lower:{lower_bound}-upper:{upper_bound}")

# Create arrays of Boolean values indicating the outlier rows
# upper_array = np.where(data['EDAD'] >= upper)[0]
# lower_array = np.where(data['EDAD'] <= lower)[0]
 
# Removing the outliers
# data=data.drop(index=upper_array)
# data=data.drop(index=lower_array).reset_index(drop=True)
# data.reset_index(drop=True)
# Print the new shape of the DataFrame

data = data[(data['EDAD'] >= lower_bound) & (data['EDAD'] <= upper_bound)]
data.reset_index(drop=True)
print("New Shape: ", data.shape)

lower:-4.0-upper:68.0
New Shape:  (4111, 11)


In [25]:
data['EDAD'].max()

50.0

In [26]:
Q1 = data['INGRE'].quantile(0.25)
Q3 = data['INGRE'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR
print(f"lower:{lower_bound} - upper:{upper_bound}")

# Create arrays of Boolean values indicating the outlier rows
# upper_array = np.where(data['INGRE'] >= upper)[0]
# lower_array = np.where(data['INGRE'] <= lower)[0]
 

# # Removing the outliers
# data=data.drop(index=upper_array)
# data=data.drop(index=lower_array).reset_index(drop=True)
# data.reset_index(drop=True)

data = data[(data['INGRE'] >= lower_bound) & (data['INGRE'] <= upper_bound)]
data.reset_index(drop=True)

# Print the new shape of the DataFrame
print("New Shape: ", data.shape)

lower:9934.5 - upper:38098.5
New Shape:  (3605, 11)


In [27]:
Q1 = data['HIJOS'].quantile(0.25)
Q3 = data['HIJOS'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

print(f"lower:{lower_bound} - upper:{upper_bound}")

# # Create arrays of Boolean values indicating the outlier rows
# upper_array = np.where(data['HIJOS'] >= upper)[0]
# lower_array = np.where(data['HIJOS'] <= lower)[0]
 

# # Removing the outliers
# data=data.drop(index=upper_array)
# data=data.drop(index=lower_array).reset_index(drop=True)
# data.reset_index(drop=True)

data = data[(data['HIJOS'] >= lower_bound) & (data['HIJOS'] <= upper_bound)]
data.reset_index(drop=True)

# Print the new shape of the DataFrame
print("New Shape: ", data.shape)

lower:-0.5 - upper:3.5
New Shape:  (3312, 11)


In [28]:
plot_continuos_histogram_matrix(data,var=None,save='report\img/continuos_hist_noOutliers.png')

In [29]:
data=data.reset_index(drop=True)

In [30]:
X=data.drop(columns=["RIESGO","TIPCRED"])
y=data['RIESGO']

In [31]:
X.head()

,EDAD,INGRE,GENERO,ECIVIL,HIJOS,NUMTDC,FPAGO,HIPOTECA,CREDITOS
0,29.0,38097.0,m,Soltero,0,1,Mensual,s,0
1,45.0,37895.0,f,Soltero,0,1,Mensual,s,0
2,34.0,37885.0,m,Soltero,0,1,Mensual,s,0
3,42.0,37837.0,m,Casado,1,2,Mensual,s,1
4,37.0,37810.0,f,Casado,1,2,Mensual,s,0


In [32]:
y.head()

0    Bueno
1    Bueno
2     Malo
3    Bueno
4     Malo
Name: RIESGO, dtype: object

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer


from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import pickle as pkl
import joblib

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [34]:
def pass_cols(x):
    return x
numeric_transformer = Pipeline(steps=[("imputer", FunctionTransformer(pass_cols))])

In [35]:
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(sparse_output=False)),
    ]
)

In [36]:
numeric_features = make_column_selector(dtype_exclude="object")(X)
categorical_features = make_column_selector(dtype_include="object")(X)

In [37]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [38]:
from sklearn.ensemble import AdaBoostClassifier

random_forest=RandomForestClassifier(n_estimators=40,
                                     max_depth=15, 
                                     random_state=20)

adaBoost = AdaBoostClassifier(n_estimators=100, random_state=0)


Preprocess_Pipeline = Pipeline([("Preprocessor", preprocessor)])

train_pipeline=Pipeline([("Preprocessor",Preprocess_Pipeline),
                        ("Model",adaBoost)],
                        verbose=True)

In [39]:
train_pipeline

Pipeline(steps=[('Preprocessor',
                 Pipeline(steps=[('Preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    FunctionTransformer(func=<function pass_cols at 0x0000024C4D4FCD30>))]),
                                                                   ['EDAD',
                                                                    'INGRE',
                                                                    'HIJOS',
                                                                    'NUMTDC',
                                                                    'CREDITOS']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('encoder',
                                                                                    OneHotEncoder(sparse_output=False))]),
                                                                   ['GENERO',
                                                                    'ECIVIL',
                                                                    'FPAGO',
                                                                    'HIPOTECA'])]))])),
                ('Model',
                 AdaBoostClassifier(n_estimators=100, random_state=0))],
         verbose=True)

In [40]:
train_pipeline.fit(X_train,y_train)

[Pipeline] ...... (step 1 of 2) Processing Preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing Model, total=   1.2s


Pipeline(steps=[('Preprocessor',
                 Pipeline(steps=[('Preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    FunctionTransformer(func=<function pass_cols at 0x0000024C4D4FCD30>))]),
                                                                   ['EDAD',
                                                                    'INGRE',
                                                                    'HIJOS',
                                                                    'NUMTDC',
                                                                    'CREDITOS']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('encoder',
                                                                                    OneHotEncoder(sparse_output=False))]),
                                                                   ['GENERO',
                                                                    'ECIVIL',
                                                                    'FPAGO',
                                                                    'HIPOTECA'])]))])),
                ('Model',
                 AdaBoostClassifier(n_estimators=100, random_state=0))],
         verbose=True)

In [59]:
# Predict
target_names = ["Bueno", "Malo","Medio"]
y_pred=train_pipeline.predict(X_train)

# Clasification report
report=classification_report(y_train, y_pred, target_names=target_names,digits=2,output_dict=True)

In [60]:
# Plot
report.update({"accuracy": {"precision": None, "recall": None, "f1-score": report["accuracy"], "support": report['macro avg']['support']}})
report=pd.DataFrame.from_dict(report).round(2)
report=report.fillna('')


fig = ff.create_table(report.T, index=True)
fig.update_layout(
    title_text="Classification report (Train)",
    margin = {'t':50, 'b':10,'l':10,'r':10},
    height=300,
    width=550,
)

pio.write_image(fig, 
                'report\img/train_report.png',
                scale=2)
fig.show()

In [58]:
y_train.value_counts()

RIESGO
Medio    1738
Malo      592
Bueno     319
Name: count, dtype: int64

In [61]:
y_pred=train_pipeline.predict(X_test)

# Classification report
report=classification_report(y_test, y_pred, target_names=target_names,digits=2,output_dict=True)

# Plot
report.update({"accuracy": {"precision": None, "recall": None, "f1-score": report["accuracy"], "support": report['macro avg']['support']}})
report=pd.DataFrame.from_dict(report).round(2)
report=report.fillna('')

fig = ff.create_table(report.T, index=True)
fig.update_layout(
    title_text="Classification report (Test)",
    margin = {'t':50, 'b':10,'l':10,'r':10},
    height=300,
    width=550,
)

pio.write_image(fig, 
                'report\img/test_report.png',
                scale=2)
fig.show()

In [65]:
cm = confusion_matrix(y_test, y_pred)
cm=pd.DataFrame(cm,columns=target_names,index=target_names)
cm

,Bueno,Malo,Medio
Bueno,44,17,33
Malo,5,81,55
Medio,11,20,397


In [53]:
cm.T

,Medio,Malo,Bueno
Medio,44,5,11
Malo,17,81,20
Bueno,33,55,397


In [74]:
fig = px.imshow(cm,text_auto=True,template='plotly_white')
fig.update_layout(
    title=f'<b>Confusion Matrix<br> </b>',
    width=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_title="Predicted",
    yaxis_title="Real"
)

pio.write_image(fig, 
                'report\img/cm.png',
                scale=2)
fig.show()

In [ ]:
joblib.dump(random_forest, r"random_forest.joblib") 

In [44]:
joblib.dump(train_pipeline, r"pipeline.joblib")

['pipeline.joblib']

In [ ]:
data.head(5)

In [ ]:
data['RIESGO']=data['RIESGO'].replace(['Malo', 'Medio', 'Bueno'],
                                      [0,1,2])

In [ ]:
data.head(5)

In [ ]:
data.index.name='id'

In [ ]:
data.head(5)

In [ ]:
data = data.rename(columns={'RIESGO': 'target'})

In [ ]:
data.head(5)

In [ ]:
data.to_csv("clean_data.csv")

In [ ]:
data.isnull().sum()

In [ ]:
import pandas as pd

data=pd.read_csv('clean_data.csv')

data.target.value_counts()